In [2]:
from scrapybara import Scrapybara

from web_recorder.recorder import Recorder

In [ ]:
SCRAPYBARA_API_KEY = "<api_key>"
client = Scrapybara(api_key=SCRAPYBARA_API_KEY)

instance = client.start_browser()

live_web_url = instance.get_stream_url().stream_url
cdp_url = instance.get_cdp_url().cdp_url
print(f"`🔍 Live View Link: {live_web_url}")

`🔍 Live View Link: https://api.proxy.scrapybara.com/v1/instance/s-fcd6992e/stream


In [4]:
recorder = Recorder(
  cdp_url=cdp_url,
)

recording = await recorder.record()

Task ID: 3ffcd67d-0244-4cbd-a7b1-de6f7a889195
recording browser logs unsupported url: about:blank found, won't record
recording browser logs Starting rrweb recording
recording browser logs window is iframe, won't record
recording browser logs window is iframe, won't record
recording browser logs window is iframe, won't record
recording browser logs send events called
recording browser logs Failed to load resource: the server responded with a status of 404 ()
recording browser logs send events called
recording browser logs send events called
recording browser logs Starting rrweb recording
recording browser logs window is iframe, won't record
recording browser logs send events called
recording browser logs Blocked attempt to show a 'beforeunload' confirmation panel for a frame that never had a user gesture since its load. https://www.chromestatus.com/feature/5082396709879808
recording browser logs send events called
recording browser logs Starting rrweb recording
recording browser logs w

In [ ]:
recording.export(
  path="../browser_recordings/save.json",
)

In [ ]:
recording = recorder.import("../browser_recordings/save.json")

In [16]:
# from playwright.async_api import async_playwright

#TODO fix this (still doesn't work)
from playwright.sync_api import Playwright, sync_playwright, expect


no_automation_args = [
    "--no-sandbox",
    "--disable-blink-features=AutomationControlled",
    "--disable-infobars",
    "--disable-background-timer-throttling",
    "--disable-popup-blocking",
    "--disable-backgrounding-occluded-windows",
    "--disable-renderer-backgrounding",
    "--disable-window-activation",
    "--disable-focus-on-load",
    "--no-first-run",
    "--no-default-browser-check",
    "--no-startup-window",
    "--window-position=0,0",
    "--disable-site-isolation-trials",
    "--disable-features=IsolateOrigins,site-per-process",
]


# with async_playwright() as p:
p = sync_playwright().start()
browser = p.chromium.launch(headless=False, args=no_automation_args)
recording.replay(browser)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.